### Context Manager in Python

Python came out with a special new keyword several years ago in Python 2.5 that is known as the **with statement**. This new keyword allows a developer to create context managers. But wait! What’s a context manager? They are handy constructs that allow you to set something up and tear something down automatically. For example, you might want to open a file, write a bunch of stuff to it and then close it. This is probably the classic example of a context manager. In fact, Python creates one automatically for you when you open a file using the with statement

https://docs.python.org/3/library/contextlib.html

In [1]:
with open(path, 'w') as f_obj:
    f_obj.write(some_data)

NameError: name 'path' is not defined

Back in Python 2.4, you would have to do it old fashioned way:

In [2]:
f_obj = open(path,'w')
f_obj.wirte(some_data)
f_obj.close()

NameError: name 'path' is not defined

The way this works under the covers is by using some of Python’s magic methods: __enter__ and __exit__. Let’s try creating our own context manager to demonstrate how this all works!

### Creating a Context Manager class

Rather than rewrite Python’s open method here, we’ll create a context manager that can create a SQLite database connection and close it when it’s done. Here’s a simple example:

In [3]:
import sqlite3

class DataConn:
    """"""
    
    def __init__(self,db_name):
        """Constructor"""
        self.db_name = db_name
    
    def __enter__(self):
        """
        Open the database connection
        """
        self.conn = sqlite3.connect(self.db_name)
        return self.conn
    
    def __exit__(self,exc_type, exc_val, exc_tb):
        """
        Close the connection
        """
        self.conn.close()
        if exc_val:
            raise
            
if __name__ == '__main__':
    db = "test.db"
    with DataConn(db)as conn:
        cursor = conn.cursor()

In the code above, we created a class that takes a path to a SQLite database file. The __enter__ method executes automatically where it creates and returns the database connection object. Now that we have that, we can create a cursor and write to the database or query it. When we exit the with statement, it causes the __exit__ method to execute and that closes the connection.

Let’s try creating a context manager using another method

### Creating a Context Manager using contextlib

Python 2.5 not only added the with statement, but it added the contextlib module. This allows us to create a ContextManager using contextlib's contextmanager function as a decorator. Let's try to create a context manager that opens and closes a file after all.


In [4]:
from contextlib import contextmanager

@contextmanager
def file_open(path):
    try:
        f_obj = open(path,'w')
        yield f_obj
    except OSError:
        print("We had an error!")
    finally:
        print("Closing the file")
        f_obj.close()

if __name__ == '__main__':
    with file_open('test.txt')as f_obj:
        f_obj.write("Testing context managers")

Closing the file


Here we just import **contextmanager** from **contextlib** and decorate our file_open function with it. This allows us to call file_open using Python's with statement. In our function, we open the file and then yield it out so the calling function can use it.

Once the **with** statement ends, control returns back to the file_open function and it continues with the code following the yeild statement. That caused the finally statement to execute, which closed the file. If we happen to have an **OSError** while working with the file, it gets caught and finally statement still closes the file handler.

### contextlib.closing(thing)

The contextlib module comes handy with some other utilities. The first one is **closing** class which will close the thing upon the completion of code block.

In [5]:
from contextlib import contextmanager

@contextmanager
def closing(db):
    try:
        yield db.conn()
    finally:
        db.close()

Basically, what we 're doing is creating a closing function that is wrapped in a contextmanager. This is the equivalent of what closing class does. The difference is that instead of decorator, we can use **closing** class itself in our **with** statement. Let's look at this:

In [6]:
# notice, it is importing closing .. 
from contextlib import closing
from urllib.request import urlopen

with closing(urlopen('http://www.google.com')) as webpage:
    for line in webpage:
        print(line)
        # don't flood the notebook
        break 
        

b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en-IN"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="ugr0ZxmvdY1fy4kOy9TaEA==">(function(){window.google={kEI:\'N6JRX6TgCMvG4-EPqc-m6AU\',kEXPI:\'0,18167,183995,1151585,5662,730,224,3657,1448,206,3204,10,1226,364,1499,611,206,383,246,5,1129,225,648,653,2799,133,5,176,3,373,677,91,40,153,735,130,116,206,245,506,114,82,31,26,103,7,658,1119056,1197760,303202,26305,13677,4855,32691,15248,867,28684,9188,8384,4859,1361,9291,3025,3892,845,5,11033,1808,4020,978,7932,5296,2974,873,38,4154,6430,1141,6291,1221,5875,4516,1399,1379,919,2277,8,87,2709,1593,1279,2212,530,149,1103,841,516,1522,4258,312,1132,4,3,2669,2023,544,1233,143,377,1947,2229,93,328,1284,16,2927,2247,1812,1787,3228,2844,7,5599,469,6286,4454,642,2450,3684,1742,4929,108,2854,553,908,2,941,2614,2397,7

In this example, we open a url page but wrap it with our closing class. This will cause the handle to the web page to be closed once we fall out of the with statement’s code block.

### contextlib.suppress(*exceptions)

Another handy tool is **suppress** class which was added in Python 3.4. The idea behind the context manager utility is that it can suppress any number of exceptions. Let's say we want to ignore the **FileNotFoundError** exception. 

In [7]:
with open('fauxfile.txt') as fobj:
    for line in fobj:
        print(line)

FileNotFoundError: [Errno 2] No such file or directory: 'fauxfile.txt'

As we can see, this context manager doesn't handle the exception. We can ignore this error like this:

In [8]:
from contextlib import suppress

with suppress(FileNotFoundError):
    with open('fauxfile.txt') as fobj:
        for line in fobj:
            print(line)

Here we import **suppress** and pass it the exception that we want to ignore. File doesn't exists, but an error is also not raised. It should be noted that this contextmanager is **reentrant**

### contextlib.redirect_stdout / redirect_stderr

The contextlib has couble of neat tools to redirecting stdout and stderr that we added in Python 3.4 and Python 3.5 respectively. Before this tools has been added, it is done like this :

In [ ]:
import sys 

path = 'text.txt'
with open(path,'w') as fobj:
    sys.stdout = fobj
    help(sum)

with the contextlib module, you can do the following:


In [ ]:
from contextlib import redirect_stdout

path = 'text1.txt'
with open(path,'w') as fobj:
    with redirect_stdout(fobj):
        help(redirect_stdout)

In both these examples, we are redirecting stdout to a file. When we call Python's **help**, instead of printing to stdout, it gets saved directly to the file. You could also redirect stdout to some kind of buffer or text control type widget from a user interface toolkit like Tkinter or wxPython

### ExitStack

**ExitStack** is a context manager that will allow you to easily programmatically combine other context managers and cleanup functions. It sounds kind of confusing at first, so let's at an example from the Python documentation to help us understand this idea a bit better:

In [ ]:
from contextlib import ExitStack

with ExitStack() as stack:
    file_objects = [stack.enter_context(open(filename))
    for filename in filenames]

This code basically creates a series of context managers inside the list comprehension. The ExitStack maintains a stack of registered callbacks that it will call in reverse order when the instance it closed, which happens when we exit of bottom of the **with** statement.

There are bunch of neat examples in Python documentation for contextlib where you can learn about topics like following:
+ Catching exceptions from __enter__ methods
+ Supporting a variable number of context managers 
+ Replacing any using try-finally
+ and much more! 

#### TODO : add these examples here ..

### Reentrant Context Managers

Most context managers that you create will be written such that they can only be used once with  **with** statement. for e.g :

In [ ]:
from contextlib import contextmanager

@contextmanager
def single():
    print('Yielding')
    yield
    print('Existing context manager')
context = single()

with context:
    pass

#Yielding
#Exiting context manager

with context:
    pass

#Traceback (most recent call last):
#  File "/usercode/__ed_file.py", line 14, in <module>
# with context
#  File "/usr/local/lib/python3.5/contextlib.py", line 61, in __enter__
#    raise RuntimeError("generator didn't yield") from None
#RuntimeError: generator didn't yield

Exception in thread IPythonHistorySavingThread:
Traceback (most recent call last):
  File "C:\Users\aviator\anaconda3\lib\site-packages\IPython\core\history.py", line 780, in writeout_cache
    self._writeout_input_cache(conn)
  File "C:\Users\aviator\anaconda3\lib\site-packages\IPython\core\history.py", line 764, in _writeout_input_cache
    (self.session_number,)+line)
sqlite3.IntegrityError: UNIQUE constraint failed: history.session, history.line

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aviator\anaconda3\lib\site-packages\IPython\core\history.py", line 834, in run
    self.history_manager.writeout_cache(self.db)
  File "<C:\Users\aviator\anaconda3\lib\site-packages\decorator.py:decorator-gen-23>", line 2, in writeout_cache
  File "C:\Users\aviator\anaconda3\lib\site-packages\IPython\core\history.py", line 58, in needs_sqlite
    return f(self, *a, **kw)
  File "C:\Users\aviator\anaconda3\lib\site-packag

Here we create an instance of our context manager and try running it twice with Python’s with statement. The second time it runs, it raises a **RuntimeError**.

But what if we wanted to be able to run the context manager twice? Well we’d need to use one that is “reentrant”. Let’s use the **redirect_stdout** context manager that we used before!

In [1]:
from contextlib import redirect_stdout
from io import StringIO

stream = StringIO()
write_to_stream = redirect_stdout(stream)
with write_to_stream:
    print('Write something to the stream')
    with write_to_stream:
        print('Write something else to the stream')

print(stream.getvalue())

Write something to the stream
Write something else to the stream



Here we create a nested context manager where they both write to StringIO, which is an in-memory text stream. The reason this works instead of raising a RuntimeError like before is that redirect_stdout is reentrant and allows us to call it twice. Of course, a real world example would be much more complex with more functions calling each other. Please also note that reentrant context managers are not necessarily thread-safe. Read the documentation before trying to use them in a thread.

### Wrapping Up
Context managers are a lot of fun and come in handy all the time. I use them in my automated tests all the time for opening and closing dialogs, for example. Now you should be able to use some of Python’s built-in tools to create your own context managers. Be sure to take the time to read the Python documentation on contextlib as there are lots of additional information that is not covered in this chapter. Have fun and happy coding!